In [ ]:
import os
from openai import OpenAI
import chromadb
from chromadb.utils import embedding_functions

# 配置阿里云 Qwen 的 Key
DASHSCOPE_API_KEY = "sk-你自己的Key"

# --- Step 1: 初始化 OpenAI 客户端 (连接 Qwen) ---
# 这就是你想要的：用 OpenAI 原生客户端
client = OpenAI(
    api_key=DASHSCOPE_API_KEY,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

prompt = "webrtc三部曲是？"

# 原生 ChatCompletion 调用
response = client.chat.completions.create(
    model="qwen-max",
    messages=[
        {"role": "system", "content": "你是一个乐于助人的AI专家。"},
        {"role": "user", "content": prompt}
    ],
    temperature=0.1
)

answer = response.choices[0].message.content
print("-" * 30)
print("最终回答：")
print(answer)
print("-" * 30)


------------------------------
最终回答：
WebRTC（Web Real-Time Communication）三部曲通常指的是实现一个基本的点对点通信应用所需要的三个关键步骤或技术组件。这三个步骤分别是：

1. **信令(Signaling)**：这是建立连接前的第一步，主要目的是让两个客户端能够交换必要的信息以开始直接通信。这些信息包括但不限于网络地址、端口等。信令本身并不属于WebRTC标准的一部分，而是需要开发者自行选择合适的传输方式来实现，比如使用WebSocket、HTTP请求或者其他任何可以用来传递消息的方法。

2. **媒体协商(Media Negotiation)**：一旦通过信令通道建立了初步联系，接下来就需要进行媒体协商了。这一步骤涉及到双方就将要使用的音频/视频编解码器类型、分辨率以及其他相关参数达成一致。在WebRTC中，这个过程是通过SDP（Session Description Protocol）来完成的，它允许参与者描述他们的多媒体会话能力，并且最终确定共同支持的最佳配置。

3. **ICE/STUN/TURN (NAT Traversal)**：最后一个阶段是为了确保即使在网络地址转换(NAT)环境下也能成功建立连接。这里涉及到的技术有：
   - **ICE (Interactive Connectivity Establishment)**: 一种框架，用于发现和测试候选者之间的连通性。
   - **STUN (Session Traversal Utilities for NAT)**: 协议帮助设备获取其公网IP地址及端口号，从而绕过简单的NAT。
   - **TURN (Traversal Using Relays around NAT)**: 当直接P2P连接不可行时，作为备用方案提供中继服务，使得数据可以通过第三方服务器转发给对方。

这三个步骤共同作用，使得WebRTC能够在各种复杂的网络环境中实现高质量的实时音视频交流。
------------------------------


In [ ]:
import os
import glob
import uuid
import time # 引入时间库，用于失败重试等待
from tqdm import tqdm
import chromadb
from openai import OpenAI
from chromadb.utils import embedding_functions
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

# ================= 配置区域 =================
DOCS_DIR = "./my_docs"
CHROMA_PATH = "./chroma_pure_db"

# *** 修正点 1: 降低入库批次大小，适应阿里云限制 ***
BATCH_SIZE = 10 

# ================= 1. 初始化 =================
client = OpenAI(
    api_key=DASHSCOPE_API_KEY,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

# *** Embedding 适配器 ***
class QwenEmbeddingFunction(chromadb.EmbeddingFunction):
    def __init__(self, api_key):
        super.__init__()

    def __call__(self, input: list[str]) -> list[list[float]]:
        # 阿里云限制每次最多 10-25 条，我们保守一点，设为 10
        # 这样无论外面传进来多少，这里都会切成小块安全发送
        safe_batch_size = 10
        all_embeddings = []
        
        # 将 input 列表切片处理
        for i in range(0, len(input), safe_batch_size):
            batch = input[i : i + safe_batch_size]
            
            # 简单的重试机制 (防止网络抖动)
            max_retries = 3
            for attempt in range(max_retries):
                try:
                    # 必须把换行符去掉，这也是embedding的一个常见坑，防止语义干扰
                    clean_batch = [text.replace("\n", " ") for text in batch]
                    
                    response = client.embeddings.create(
                        model="text-embedding-v3",
                        input=clean_batch,
                        encoding_format="float"
                    )
                    # 收集结果
                    batch_embeddings = [data.embedding for data in response.data]
                    
                    #append是将参数作为一个整体添加到list中，输入【hello】["world", "hello"]
                    #extend是将参数看作是一组数据，把里边的每一项添加到list中【hello】['a','b','h','e',...]
                    all_embeddings.extend(batch_embeddings)
                    break # 成功则跳出重试循环
                except Exception as e:
                    if attempt < max_retries - 1:
                        time.sleep(1) # 等1秒再试
                        continue
                    else:
                        # 如果3次都失败，打印严重错误，但不要让整个程序崩溃
                        # 注意：这里如果返回空，Chroma 还是会报错，但至少你能看到日志
                        print(f"\n[Error] Embedding API Failed after retries: {e}")
                        # 填充零向量占位，防止程序崩溃 (可选，或者直接抛出异常)
                        # 这里我们选择抛出异常让上层知道
                        raise e

        return all_embeddings

# 初始化数据库
chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)
collection = chroma_client.get_or_create_collection( #创建一个表
    name="markdown_knowledge_base",
    embedding_function=QwenEmbeddingFunction()
)

# ================= 2. 核心处理逻辑 (保持不变，微调参数) =================

def process_and_index_files():
    md_files = glob.glob(os.path.join(DOCS_DIR, "*.md"))
    print(f"发现 {len(md_files)} 个Markdown文件，准备处理...")

    # 这是一个映射列表。它告诉程序：“当你看到 # 时，把它当作一级标题（H1）；看到 ## 时，当作二级标题（H2）...”。
    # 目的：不仅仅是切分，还会将标题内容作为**元数据（Metadata）**保留下来。
    #   比如，切分出的片段会带有一个标签 {'H1': '用户手册', 'H2': '安装指南'}，这样模型检索时就知道这段文字属于哪个章节。
    headers_to_split_on = [("#", "H1"), ("##", "H2"), ("###", "H3")]
    md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    
    # 作用：它会按照上面定义的标题层级，将 Markdown 文本“物理”切断。
    # 结果：原本的一个长文档，会根据标题被拆分成多个逻辑段落。
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )

    batch_documents = []
    batch_metadatas = []
    batch_ids = []

    for file_path in tqdm(md_files, desc="Processing Files"):
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()
            
            filename = os.path.basename(file_path)
            header_splits = md_splitter.split_text(content)
            final_splits = text_splitter.split_documents(header_splits)

            for split in final_splits:
                meta = split.metadata.copy()
                meta["source"] = filename 
                
                # 过滤掉空内容
                if not split.page_content.strip():
                    continue

                batch_documents.append(split.page_content)
                batch_metadatas.append(meta)
                batch_ids.append(f"{filename}_{uuid.uuid4().hex[:8]}")

                # 提交判断
                if len(batch_documents) >= BATCH_SIZE:
                    collection.upsert(
                        documents=batch_documents,
                        metadatas=batch_metadatas,
                        ids=batch_ids
                    )
                    batch_documents = []
                    batch_metadatas = []
                    batch_ids = []

        except Exception as e:
            print(f"\n跳过文件 {file_path}: {e}")

    # 处理尾部数据
    if batch_documents:
        collection.upsert(
            documents=batch_documents,
            metadatas=batch_metadatas,
            ids=batch_ids
        )
    
    print(f"\n全部处理完成！数据已存入 {CHROMA_PATH}")
    print(f"当前数据库总数: {collection.count()}")

if __name__ == "__main__":
    if not os.path.exists(DOCS_DIR):
        os.makedirs(DOCS_DIR)
        print(f"请将你的 Markdown 文件放入 {DOCS_DIR} 文件夹中。")
    else:
        # 清理旧数据 (可选：如果你想每次重新跑都清空库的话，取消注释下面这行)
        chroma_client.delete_collection("markdown_knowledge_base")
        collection = chroma_client.get_or_create_collection("markdown_knowledge_base", embedding_function=QwenEmbeddingFunction())
        
        process_and_index_files()

/var/folders/wc/dk4btz8500gfxc94n2218h140000gn/T/ipykernel_13372/3837800524.py:70: DeprecationWarning: The class QwenEmbeddingFunction does not implement __init__. This will be required in a future version.
  embedding_function=QwenEmbeddingFunction()
/var/folders/wc/dk4btz8500gfxc94n2218h140000gn/T/ipykernel_13372/3837800524.py:144: DeprecationWarning: The class QwenEmbeddingFunction does not implement __init__. This will be required in a future version.
  collection = chroma_client.get_or_create_collection("markdown_knowledge_base", embedding_function=QwenEmbeddingFunction())


发现 102 个Markdown文件，准备处理...


Processing Files: 100%|██████████| 102/102 [01:57<00:00,  1.15s/it]



全部处理完成！数据已存入 ./chroma_pure_db
当前数据库总数: 1484


In [11]:
import chromadb

chroma_client = chromadb.PersistentClient(path="./chroma_pure_db")
collection = chroma_client.get_collection("markdown_knowledge_base") # 注意：这里不需要传 embedding_function，因为只读不写

print(f"当前数据库共有 {collection.count()} 条数据。")

# 随便拿一条来看看元数据对不对
peek = collection.peek(1)
print("数据样本:", peek)

当前数据库共有 1484 条数据。
数据样本: {'ids': ['OpenCV课程资料.md_6fde3ea4'], 'embeddings': array([[-0.05851695, -0.01614801, -0.06831551, ..., -0.0196167 ,
         0.05310815, -0.01203262]]), 'documents': ['---\ntitle: OpenCV课程资料\ntags:\n- OpenCV\n- 人工智能\n- AI\ncategories: AI\nabbrlink: 65de593f\ndate: 2020-05-31 00:19:12\n---  \n这里是我OpenCV课程的相关资料，后面还会不断补充...\nAI\n<!-- more -->'], 'uris': None, 'included': ['metadatas', 'documents', 'embeddings'], 'data': None, 'metadatas': [{'source': 'OpenCV课程资料.md'}]}


In [ ]:
client = OpenAI(
    api_key=DASHSCOPE_API_KEY,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

chroma_client = chromadb.PersistentClient(path="./chroma_pure_db")
collection = chroma_client.get_collection(
    name="markdown_knowledge_base",
    embedding_function=QwenEmbeddingFunction() 
)

# --- Step 4: 检索 (Retrieval) ---
user_query = "webrtc三部曲是什么？"

# 执行相似度检索
# 它找出距离最近（最相似）的 n 个文档返回给你。
results = collection.query(
    query_texts=[user_query],
    n_results=1
)

# 提取检索到的文本 (Chroma 返回的是个复杂的字典)
retrieved_context = results['documents'][0][0]
print(f"\n检索到的背景信息: {retrieved_context}")

# --- Step 5: 生成 (Generation) ---
# 手写 Prompt，没有任何黑盒
prompt = f"""
你是一个助手。请根据下面的【参考信息】回答用户问题。

【参考信息】：
{retrieved_context}

【用户问题】：
{user_query}
"""

print("\n正在请求 Qwen 模型...")

# 原生 ChatCompletion 调用
response = client.chat.completions.create(
    model="qwen-max",
    messages=[
        {"role": "system", "content": "你是一个乐于助人的AI专家。"},
        {"role": "user", "content": prompt}
    ],
    temperature=0.1
)

answer = response.choices[0].message.content
print("-" * 30)
print("最终回答：")
print(answer)
print("-" * 30)

/var/folders/wc/dk4btz8500gfxc94n2218h140000gn/T/ipykernel_13372/3532827569.py:9: DeprecationWarning: The class QwenEmbeddingFunction does not implement __init__. This will be required in a future version.
  embedding_function=QwenEmbeddingFunction()



检索到的背景信息: **WebRTC三部曲**，这个计划渐渐的在我的脑海中浮现出来，于是2017年我出来创业了。  
WebRTC三部曲的第一部是[《WebRTC入门与实战》](https://coding.imooc.com/class/329.html)，这门课从WebRTC的应用角度来讲，主要讲WebRTC都能做什么，该怎么使用它，包括各个终端的互联互通。这门课已于2019年上线；第二部[《WebRTC流媒体服器》](https://coding.imooc.com/class/387.html)讲的是如何设计、实现一个可以高负载、大并发，并且能与WebRTC（浏览器）互通的流媒体服务器，这门课同样于2019年上线；第三部就是刚刚更新完的[《WebRTC源码深入剖析》](https://coding.imooc.com/class/532.html)。

正在请求 Qwen 模型...
------------------------------
最终回答：
WebRTC三部曲是一系列关于WebRTC技术的学习课程，旨在帮助开发者从入门到深入理解WebRTC。这三部曲包括：

1. **《WebRTC入门与实战》**：此课程专注于WebRTC的应用层面，介绍WebRTC能够实现的功能以及如何使用它来开发应用，同时涵盖了不同终端之间的互联互通问题。该课程于2019年上线。

2. **《WebRTC流媒体服务器》**：本课程侧重于教授如何设计和构建一个高性能、高并发的流媒体服务器，并确保该服务器能够与基于WebRTC技术的浏览器端良好地协同工作。同样地，这门课也在2019年推出。

3. **《WebRTC源码深入剖析》**：作为三部曲的最后一部分，这门课程提供了对WebRTC源代码的深度解析，适合那些希望深入了解WebRTC内部工作机制的技术爱好者或专业人士。这门课程是最新更新完成的。

通过这三个阶段的学习，学习者可以从基础概念逐步过渡到高级实践乃至底层原理的理解，全面掌握WebRTC相关知识和技术。
------------------------------
